In [8]:
!pip install tensorflow
!pip install tensorflow-gpu
!pip install nvidia-cudnn-cu12
!pip install Pillow

# Rodar um 
# conda install -c conda-forge cudatoolkit

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [39 lines of output]
      Traceback (most recent call last):
        File "/home/vinmelo/Documents/Deep-Learning-Package-Inspection/.venv/lib/python3.10/site-packages/packaging/requirements.py", line 36, in __init__
          parsed = _parse_requirement(requirement_string)
        File "/home/vinmelo/Documents/Deep-Learning-Package-Inspection/.venv/lib/python3.10/site-packages/packaging/_parser.py", line 62, in parse_requirement
          return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
        File "/home/vinmelo/Documents/Deep-Learning-Package-Inspection/.venv/lib/python3.10/site-packages/packaging/_parser.py", line 80, in _parse_requirement
          url, specifier, marker = _parse_requirement_details(tokenizer)
        File "/home/vinmelo/Documents/Deep-

## Modelo para Side
### Importando o Modelo pré treinado

In [2]:
from tensorflow import keras
base_model = keras.applications.VGG16(
    weights="imagenet",
    input_shape=(320,576,3),
    include_top=False
)

In [3]:
base_model.trainable = False

In [4]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.19.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Adicionando Camadas

In [5]:
from PIL import Image
import numpy as np

img_side = Image.open('./dataset-desafio/side/damaged/0185507921789_side.png')
img_array_side = np.array(img_side)

img_top = Image.open('./dataset-desafio/top/damaged/0185507921789_top.png')
img_array_top = np.array(img_top)

print("Shape side:", img_array_side.shape)
print("Shape top:", img_array_top.shape)

Shape side: (540, 960, 3)
Shape top: (540, 960, 3)


In [6]:
inputs = keras.Input(shape=(320,576,3))

x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)

outputs = keras.layers.Dense(1, activation= 'sigmoid')(x)

model = keras.Model(inputs,outputs)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 320, 576, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 10, 18, 512)    │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,715,201 (56.13 MB)

 Trainable params: 513 (2.00 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [7]:
model.compile(loss = 'binary_crossentropy', metrics=['accuracy','Precision','Recall', 'AUC'])

In [12]:
train_dt = keras.utils.image_dataset_from_directory(
    "./dataset-desafio/side",
    validation_split=0.2,
    subset="training",
    batch_size=2,
    seed=123,
    color_mode="rgb",
    image_size=(320,576),
    label_mode='binary'
)

valid_dt = keras.utils.image_dataset_from_directory(
    "./dataset-desafio/side",
    validation_split=0.2,
    subset="validation",
    batch_size=2,
    seed=123,
    color_mode="rgb",
    image_size=(320,576),
    label_mode='binary'
)

Found 180 files belonging to 2 classes.


Using 144 files for training.
Found 180 files belonging to 2 classes.
Using 36 files for validation.


In [13]:
model.fit(train_dt,
          validation_data=valid_dt,
          epochs=20)

Epoch 1/20


2025-07-03 15:05:21.498830: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{k25=2} for conv %cudnn-conv-bias-activation.49 = (f32[2,512,20,36]{3,2,1,0}, u8[0]{0}) custom-call(f32[2,512,20,36]{3,2,1,0} %bitcast.7324, f32[512,512,3,3]{3,2,1,0} %bitcast.5388, f32[512]{0} %bitcast.7364), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="functional_1/vgg16_1/block5_conv1_1/convolution" source_file="/home/vinmelo/Documents/Deep-Learning-Package-Inspection/.venv/lib/python3.10/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}


72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 601ms/step - AUC: 0.7161 - Precision: 0.7147 - Recall: 0.7173 - accuracy: 0.6836 - loss: 0.8325

2025-07-03 15:06:40.587077: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{k25=2} for conv %cudnn-conv-bias-activation.49 = (f32[2,512,20,36]{3,2,1,0}, u8[0]{0}) custom-call(f32[2,512,20,36]{3,2,1,0} %bitcast.1325, f32[512,512,3,3]{3,2,1,0} %bitcast.1332, f32[512]{0} %bitcast.1334), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="functional_1/vgg16_1/block5_conv1_1/convolution" source_file="/home/vinmelo/Documents/Deep-Learning-Package-Inspection/.venv/lib/python3.10/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}


72/72 ━━━━━━━━━━━━━━━━━━━━ 95s 869ms/step - AUC: 0.7181 - Precision: 0.7153 - Recall: 0.7183 - accuracy: 0.6850 - loss: 0.8286 - val_AUC: 0.9448 - val_Precision: 0.9375 - val_Recall: 0.6818 - val_accuracy: 0.7778 - val_loss: 0.4557
Epoch 2/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 47s 649ms/step - AUC: 0.9402 - Precision: 0.8894 - Recall: 0.8997 - accuracy: 0.8873 - loss: 0.3106 - val_AUC: 0.9318 - val_Precision: 0.9500 - val_Recall: 0.8636 - val_accuracy: 0.8889 - val_loss: 0.3410
Epoch 3/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 47s 650ms/step - AUC: 0.9886 - Precision: 0.9564 - Recall: 0.9670 - accuracy: 0.9620 - loss: 0.1324 - val_AUC: 0.9659 - val_Precision: 0.9500 - val_Recall: 0.8636 - val_accuracy: 0.8889 - val_loss: 0.2978
Epoch 4/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 47s 648ms/step - AUC: 0.9952 - Precision: 0.9865 - Recall: 0.9714 - accuracy: 0.9793 - loss: 0.0737 - val_AUC: 0.9675 - val_Precision: 0.9524 - val_Recall: 0.9091 - val_accuracy: 0.9167 - val_loss: 0.2348
Epoch 5/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 4

In [11]:
base_model.trainable = True

model.compile(optimizer=keras.optimizers.RMSprop(learning_rate= 0.00001),
              loss = 'binary_crossentropy', metrics=['accuracy','Precision','Recall', 'AUC'])

model.fit(train_dt,
          validation_data=valid_dt,
          epochs=10)

Epoch 1/10


2025-07-03 15:04:34.289770: W tensorflow/core/framework/op_kernel.cc:1857] OP_REQUIRES failed at xla_ops.cc:591 : UNKNOWN: Failed to determine best cudnn convolution algorithm for:
%cudnn-conv-bw-filter.14 = (f32[64,64,3,3]{3,2,1,0}, u8[0]{0}) custom-call(f32[4,64,320,576]{3,2,1,0} %bitcast.6514, f32[4,64,320,576]{3,2,1,0} %bitcast.6558), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", metadata={op_type="Conv2DBackpropFilter" op_name="gradient_tape/functional_1/vgg16_1/block1_conv2_1/convolution/Conv2DBackpropFilter" source_file="/home/vinmelo/Documents/Deep-Learning-Package-Inspection/.venv/lib/python3.10/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}

Original error: RESOURCE_EXHAUSTE

UnknownError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/vinmelo/Documents/Deep-Learning-Package-Inspection/.venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/vinmelo/Documents/Deep-Learning-Package-Inspection/.venv/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/vinmelo/Documents/Deep-Learning-Package-Inspection/.venv/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/vinmelo/Documents/Deep-Learning-Package-Inspection/.venv/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 211, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, i

## Modelo para top